In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [39]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [55]:
data2 = requests.get(standings_url)

In [53]:
soup = BeautifulSoup(data2.text)


In [56]:
standings_tb = soup.select("table.stats_table")[0]


IndexError: list index out of range

In [50]:
links = standings_tb.find_all('a')

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [ ]:
#get all the links in the table
links = [l.get("href") for l in links]

In [ ]:
#Look for just the links to the team pages and store them
links = [l for l in links if '/squads/' in l]

In [ ]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [ ]:
team_urls

In [ ]:
ars_url = team_urls[0] 

In [ ]:
data = requests.get(ars_url)

In [ ]:
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [ ]:
soup = BeautifulSoup(data.text)

In [ ]:
links = soup.find_all('a')

In [ ]:
links = [l.get("href") for l in links]

In [ ]:
links = [l for l in links if l and 'all_comps/shooting/' in l]



In [ ]:
#Get the html page for the corresponding team's shooting stats
data = requests.get(f"https://fbref.com{links[0]}")

In [ ]:
#read the data into a pandas df
shooting = pd.read_html(data.text, match="Shooting")[0]

type(shooting)

In [ ]:
#Remove multi-level index
shooting.columns = shooting.columns.droplevel()

In [ ]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist","FK","PK","PKatt","xG"]], on="Date" )

In [ ]:
team_data

In [ ]:
matches.shape


In [ ]:
shooting.shape

In [2]:
years = list(range(2023,2020,-1))
all_matches=[]

In [3]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [6]:

for year in years:
    
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_tb = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_tb.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")

        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist","FK","PK","PKatt","xG"]], on="Date")
        except ValueError:
            continue
        
        team_data = team_data[team_data["Comp"] == "Premier League"] 
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(10)

KeyboardInterrupt: 

In [6]:
len(all_matches)


60

In [7]:
match_df = pd.concat(all_matches)


In [8]:
match_df.columns = [c.lower() for c in match_df.columns]


In [10]:
match_df.to_csv("prem_2023.csv")